# 0. Magic

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

In [2]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
import operator

In [3]:
# tol = tolerance
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f"Near Zero: {a}"

# 2. Data

In [4]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [5]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [6]:
x_train, y_train, x_valid, y_valid = get_data()

# 3. เตรียมข้อมูล

หา mean และ standard deviation ของข้อมูล Training Set

In [7]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

Normalize คือการทำให้ข้อมูล มี mean = 0 และ standard deviation (std) = 1 ด้วยกัน ลบด้วย mean หารด้วย std

In [8]:
# x = data, m = mean, s = standard deviation
def normalize(x, m, s): 
    return (x-m)/s

In [9]:
x_train = normalize(x_train, train_mean, train_std)

Normalize ข้อมูล Validation Set ด้วย mean, std ของ Training Set เพื่อปรับให้เป็น Scale เดียวกัน

In [10]:
x_valid = normalize(x_valid, train_mean, train_std)

ลองดูค่า mean, std หลังจาก Normalize เรียบร้อยแล้ว

In [11]:
train_mean_after, train_std_after = x_train.mean(), x_train.std()
train_mean_after, train_std_after

(tensor(-7.6999e-06), tensor(1.))

In [12]:
test_near_zero(train_mean_after)

In [13]:
test_near_zero(1-train_std_after)

In [14]:
# n = จำนวน Record, m จำนวน Feature, c = จำนวน class
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

# 4. Model

เราจะสร้าง 2 Hidden Layers Neural Networks ที่มี 50 Neuron ในแต่ละ Hidden Layer 

In [15]:
# nh = number of hidden
nh = 50

## 4.1 Random Initialize

เราจะลองกำหนดค่าเริ่มต้นของ Weight และ Bias ด้วยค่า Random แบบ Normal (Gaussian)

In [16]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

ลองดูค่า mean, std ของ Weight

In [17]:
w1.mean(), w1.std()

(tensor(0.0060), tensor(1.0008))

In [18]:
# x = data, w = weight, b = bias
def lin(x, w, b): 
    return x@w + b

In [19]:
t = lin(x_train, w1, b1)

In [20]:
t.mean(), t.std()

(tensor(0.4586), tensor(27.0322))

In [21]:
def relu(x):
    return torch.clamp(x, 0)

In [22]:
t = relu(t)

In [23]:
t.mean(), t.std()

(tensor(10.8898), tensor(15.9954))